In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras import models, layers, losses, optimizers, regularizers

# Load the IMDB dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# Preparing the data
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

# Splitting the data for validation
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

# Experiment 1: Varying number of hidden layers
model_1_layer = models.Sequential([
    layers.Dense(16, activation='relu', input_shape=(10000,)),
    layers.Dense(1, activation='sigmoid')
])

model_3_layers = models.Sequential([
    layers.Dense(16, activation='relu', input_shape=(10000,)),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Experiment 2: Varying number of hidden units
model_32_units = models.Sequential([
    layers.Dense(32, activation='relu', input_shape=(10000,)),
    layers.Dense(1, activation='sigmoid')
])

model_64_units = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(10000,)),
    layers.Dense(1, activation='sigmoid')
])

# Experiment 3: Trying mse loss function
model_mse = models.Sequential([
    layers.Dense(16, activation='relu', input_shape=(10000,)),
    layers.Dense(1, activation='sigmoid')
])

# Experiment 4: Trying tanh activation function
model_tanh = models.Sequential([
    layers.Dense(16, activation='tanh', input_shape=(10000,)),
    layers.Dense(16, activation='tanh'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the models
models_list = [model_1_layer, model_3_layers, model_32_units, model_64_units, model_mse, model_tanh]
for model in models_list:
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Experiment 5: Adding dropout and L2 regularization
model_reg_drop = models.Sequential([
    layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(10000,)),
    layers.Dropout(0.5),
    layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model_reg_drop.compile(optimizer='rmsprop',
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

# Train all models
epochs = 20
history_list = []
for model in models_list:
    history = model.fit(partial_x_train,
                        partial_y_train,
                        epochs=epochs,
                        batch_size=512,
                        validation_data=(x_val, y_val),
                        verbose=0)
    history_list.append(history)

# Evaluate all models on test data
for i, model in enumerate(models_list):
    print(f"Model {i+1} Evaluation:")
    loss, accuracy = model.evaluate(x_test, y_test)
    print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Plotting the validation loss for all models
plt.figure(figsize=(10, 6))
for i, history in enumerate(history_list):
    val_loss = history.history['val_loss']
    epochs = range(1, len(val_loss) + 1)
    plt.plot(epochs, val_loss, label=f"Model {i+1}")
plt.title('Validation Loss for Different Models')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.legend()
plt.grid(True)
plt.show()
